In [1]:
import pandas as pd
import os
import glob

df = pd.read_csv('wt-N2-081015-01.csv', names=('t', 'pid', 'cid', 'x', 'y', 'z', 'name', 'sphericity', 'volume', 'phase'))
df['entity'] = 'line'

In [2]:
df.head()

,t,pid,cid,x,y,z,name,sphericity,volume,phase,entity
0,1,0,1000,31.2026,11.0956,12.5788,P0,0.837884,76238.3,N,line
1,2,1000,2000,31.4154,11.5594,12.7917,P0,0.811657,88108.6,N,line
2,3,2000,3000,32.0421,11.3223,12.5404,P0,0.854491,110026.0,N,line
3,4,3000,4000,32.5862,11.4526,12.4790,P0,0.771704,107802.0,N,line
4,5,4000,5000,33.7902,11.7388,11.4331,P0,0.815484,138333.0,N,line


In [3]:
df

,t,pid,cid,x,y,z,name,sphericity,volume,phase,entity
0,1,0,1000,31.2026,11.0956,12.5788,P0,0.837884,76238.3,N,line
1,2,1000,2000,31.4154,11.5594,12.7917,P0,0.811657,88108.6,N,line
2,3,2000,3000,32.0421,11.3223,12.5404,P0,0.854491,110026.0,N,line
3,4,3000,4000,32.5862,11.4526,12.4790,P0,0.771704,107802.0,N,line
4,5,4000,5000,33.7902,11.7388,11.4331,P0,0.815484,138333.0,N,line
...,...,...,...,...,...,...,...,...,...,...,...
1058,185,184010,185010,25.5795,19.7011,14.5596,Cp,0.657980,26864.0,N,line
1059,185,184009,185011,26.1515,53.6567,15.1773,ABala,0.562700,24160.8,N,line
1060,185,184011,185012,38.0593,21.8514,15.3465,ABprp,0.738705,140320.0,N,line
1061,185,184013,185013,32.7964,44.7435,19.8601,ABara,0.730234,95857.5,N,line


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063 entries, 0 to 1062
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   t           1063 non-null   int64  
 1   pid         1063 non-null   int64  
 2   cid         1063 non-null   int64  
 3   x           1063 non-null   float64
 4   y           1063 non-null   float64
 5   z           1063 non-null   float64
 6   name        1063 non-null   object 
 7   sphericity  1063 non-null   float64
 8   volume      1063 non-null   float64
 9   phase       1063 non-null   object 
 10  entity      1063 non-null   object 
dtypes: float64(5), int64(3), object(3)
memory usage: 91.5+ KB


In [5]:
import anndata as ad
import numpy as np

In [6]:
n_obs, n_vars = 1063, 4
nmp = df[["t", "z", "y", "x"]].values
Y = nmp
X1 = Y / np.array([1, 0.5016, 0.1015, 0.1015])
X2 = X1 * np.array([1, -1, 1, 1])
X = X2 + np.array([0, 65, 0, 0])

In [7]:
X
dfa = pd.DataFrame(X, columns=list('tzyx'), index=np.arange(n_obs, dtype=int).astype(str))
dfa

,t,z,y,x
0,1.0,39.922648,109.316256,307.414778
1,2.0,39.498206,113.885714,309.511330
2,3.0,39.999203,111.549754,315.685714
3,4.0,40.121611,112.833498,321.046305
4,5.0,42.206738,115.653202,332.908374
...,...,...,...,...
1058,185.0,35.973684,194.099507,252.014778
1059,185.0,34.742225,528.637438,257.650246
1060,185.0,34.404904,215.284729,374.968473
1061,185.0,25.406499,440.822660,323.117241


In [9]:
obs_raw = df.loc[:,["cid", "entity", "name", "sphericity", "volume"]]

obs_meta = obs_raw.rename(columns={'cid': 'id'})

col = obs_meta.loc[:, 'id']
col.transpose()

0         1000
1         2000
2         3000
3         4000
4         5000
         ...  
1058    185010
1059    185011
1060    185012
1061    185013
1062    185014
Name: id, Length: 1063, dtype: int64

In [10]:
obs_meta

,id,entity,name,sphericity,volume
0,1000,line,P0,0.837884,76238.3
1,2000,line,P0,0.811657,88108.6
2,3000,line,P0,0.854491,110026.0
3,4000,line,P0,0.771704,107802.0
4,5000,line,P0,0.815484,138333.0
...,...,...,...,...,...
1058,185010,line,Cp,0.657980,26864.0
1059,185011,line,ABala,0.562700,24160.8
1060,185012,line,ABprp,0.738705,140320.0
1061,185013,line,ABara,0.730234,95857.5


In [11]:
obsp_raw = np.loadtxt('wt-N2-081015-01-track.csv', delimiter=',', dtype=np.int64)
obsp_meta = pd.DataFrame(obsp_raw, index = col.astype(str), columns = col.astype(str))
obsp_meta

id,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,...,185005,185006,185007,185008,185009,185010,185011,185012,185013,185014
id,,,,,,,,,,,,,,,,,,,,,
1000,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3000,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4000,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5000,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
adata = ad.AnnData(X = dfa, obs = obs_meta)
adata.obsp["tracking"] = obsp_raw

/Users/kkyoda/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [13]:
print(adata)

AnnData object with n_obs × n_vars = 1063 × 4
    obs: 'id', 'entity', 'name', 'sphericity', 'volume'
    obsp: 'tracking'


In [14]:
adata.to_df().head()

,t,z,y,x
0,1.0,39.922649,109.316254,307.414764
1,2.0,39.498207,113.885712,309.511322
2,3.0,39.999203,111.549751,315.685699
3,4.0,40.121613,112.833496,321.046295
4,5.0,42.206738,115.653198,332.908386


In [15]:
adata.obs.head()

,id,entity,name,sphericity,volume
0,1000,line,P0,0.837884,76238.3
1,2000,line,P0,0.811657,88108.6
2,3000,line,P0,0.854491,110026.0
3,4000,line,P0,0.771704,107802.0
4,5000,line,P0,0.815484,138333.0


In [16]:
adata.obsp["tracking"]

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
adata.write_zarr('dyn')

/Users/kkyoda/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'entity' as categorical
/Users/kkyoda/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'name' as categorical
